In [ ]:
# imports we need + utils
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import hvplot.pandas
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from datetime import datetime
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_auc_score, roc_curve, auc,
    # plot_confusion_matrix, plot_roc_curve
)
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
# import imblearn as iblearn
# import sklearn as skl
# print(iblearn.__version__)
# print(skl.__version__)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

bold_start = "\033[1m"
bold_end = "\033[0m"

# import sklearn as sk
# print(sk.__version__)
# import imblearn as ib
# print(ib.__version__)

**Global Variables for metrics**

In [ ]:
# Global data for metrics
metrics_dict_val = {
    'XGBoost': {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0},
    'Logistic Regression': {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0},
    'Random Forest': {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0}
}

metrics_dict_test = {
    'XGBoost': {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0},
    'Logistic Regression': {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0},
    'Random Forest': {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0}
}

In [ ]:
# Exploratory Data Analysis code cell
data = pd.read_csv("resources/lending_club_loan_two.csv")


def plot_3_row_histogram(df):
    numerics_df = df.select_dtypes(include=['number'])
    print("Number of numeric cols: ", len(numerics_df.columns), " of ", len(df.columns))
    n_rows = len(numerics_df.columns) // 3
    n_rows += len(numerics_df.columns) % 3

    # Create a figure and axes with matplotlib's subplots()
    fig, axs = plt.subplots(n_rows, 3, figsize=(15, n_rows * 4))

    for idx, col in enumerate(numerics_df.columns):
        i = idx // 3
        j = idx % 3

        sns.histplot(numerics_df[col], ax=axs[i][j], kde=True)
        axs[i][j].set_title(f"Distribution of {col}")

    # Remove empty subplots
    if len(numerics_df.columns) % 3:
        for j in range(len(numerics_df.columns) % 3, 3):
            fig.delaxes(axs[i][j])

    plt.tight_layout()
    plt.show()


def plot_label_distribution(df, label):
    values = df[label].value_counts()
    values.plot(kind='bar', figsize=(5, 3))
    plt.title('Label values distribution')
    plt.show()


def plot_correlation_matrix(df):
    numeric_df = df.select_dtypes(include=['number'])
    plt.figure(figsize=(10, 8))
    sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
    plt.title('Correlation Matrix')
    plt.show()


def percentage_of_missing_values(df):
    missing_columns = df.columns[data.isna().any()].to_list()
    missing_values = df[missing_columns].isna().mean() * 100
    missing_values = missing_values.apply(lambda x: f'{x: .2f}%')

    # missing_df = pd.DataFrame({'Column': missing_columns, 'Missing %': missing_values})
    missing_df = pd.DataFrame({'Missing %': missing_values})
    print("Percentage of missing values")
    display(missing_df)


def detect_outliers_zscore(df, column, threshold=3):
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    outliers = df[z_scores > threshold].index
    return outliers


def show_outliers(df):
    outlier_data = []
    numerics_df = df.select_dtypes(include=['number'])
    for col in numerics_df.columns:
        ol = detect_outliers_zscore(numerics_df, col)
        outlier_data.append({'Column': col, 'Outlier Count': len(ol)})
    outliers_df = pd.DataFrame(outlier_data)
    print("Outliers found in each feature")
    display(outliers_df.sort_values(by='Outlier Count', ascending=False))


def show_unique_values_categorical_features(df):
    categorical_cols = data.select_dtypes(include=['object'])
    unique_categories = []
    for col in categorical_cols.columns:
        unique_categories.append({'Column': col, 'Unique Count': data[col].nunique()})
    unique_categories_df = pd.DataFrame(unique_categories)
    print("Count of unique values in categorical features")
    display(unique_categories_df.sort_values(by='Unique Count', ascending=False))


def feature_importance_random_forest(x, y):
    # Train a Random Forest model
    rf = RandomForestClassifier()
    rf.fit(x, y)

    # Get feature importance
    importance = rf.feature_importances_

    # Create a DataFrame to visualize importance
    feature_importance = pd.DataFrame({'Feature': x.columns, 'Importance': importance})
    feature_importance = feature_importance.sort_values('Importance', ascending=False)

    print(feature_importance)


def pub_rec(number):
    if number == 0.0:
        return 0
    else:
        return 1


def mort_acc(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number


def pub_rec_bankruptcies(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number


def eda():
    #label distribution
    plot_label_distribution(data, 'loan_status')

    # feature distribution
    plot_3_row_histogram(data)

    #correlation matrix
    plot_correlation_matrix(data)

    # missing values
    percentage_of_missing_values(data)

    # outliers
    show_outliers(data)

    # show unique values for each categorical feature
    show_unique_values_categorical_features(data)


# Exploratory Data Exploration

In [ ]:
# call exploratory data analysis methods
# eda()

## EDA Observations

## Distributions:
#### Right Skewed: 
* *loan_amnt*, *int_rate*, *installment*, *annual_inc*, *dti*, *open_acc*, *pub_rec*, *revol_bal*, *total_acc* 
* These variables have a long tail on the right, indicating a concentration of values at the lower end with a few very high values.

#### Approximately Normal: 
* *revol_util*
* This distribution resembles a bell curve, suggesting a symmetric distribution of values around the mean.

#### Highly Imbalanced: 
* `mort_acc, pub_rec_bankruptcies` 
* These variables have a very uneven distribution, with a large majority of values concentrated in a single category.

### Preprocessing Strategies:
1. Addressing the Imbalance in label instances:
    - Oversampling using SMOTE and ADASYN: From the bar graph above relating to Fully Paid vs Charged-Off, we can see a significant imbalance in label instance that need to be addressed.
    - We're using both Synthetic Minority Sampling and Adaptive Synthetic Sampling to experiment and cross-validate which is better since SMOTE might not perform well with complex distributions
2. Handling Skewness:
    - We chose to cap the extreme values at a certain percentile to reduce the impact of these outliers without removing them entirely. 
3. Normalization/Standardization:
    - We shall standardize `revol_util` since it's close to normal distribution. This will help improve model performance.
    - All other variables to are going to be normalized using min-max scaling to rescale them to a range between 0 and 1. This will prevent features with larger variations from biasing the models we train later on.
4. Because `pub_rec, mort_acc, pub_rec_bankruptcies` are heavily right-skewed and have most of their values as 0, we're choosing to simplify them into binary e.g. 0 - no mort_acc, 1 - has mort_acc  
5. Address feature needs to be augmented/engineered into zip_code because there are too many variations for it to be encoded effeciently.

# Data Preprocessing

The goal here is to do the following:

* Impute missing values
* Remove repeating and irrelevant features
* Encode categorical features into numerical format
* Oversample the minority class to increase its number of instances
* Change some features into variations of themselves (feature engineer)

In [ ]:
# Convert features to simple binary 
data['pub_rec'] = data.pub_rec.apply(pub_rec)
data['mort_acc'] = data.mort_acc.apply(mort_acc)
data['pub_rec_bankruptcies'] = data.pub_rec_bankruptcies.apply(pub_rec_bankruptcies)
data['loan_status'] = data.loan_status.map({'Fully Paid': 1, 'Charged Off': 0})

# one-hot encode categories we found as categorical
one_hot_encode_cols = ['verification_status',  'initial_list_status',
                       'application_type']
data = pd.get_dummies(data, columns=one_hot_encode_cols, drop_first=True)

# the shape of the data
print(f"The shape of the dataset (instances, features): {data.shape}")

### 2.2 Converting categorical string features into numerical formats

In [ ]:
data.term.unique()

In [ ]:
term_values = {' 36 months': 36, ' 60 months': 60}
data['term'] = data.term.map(term_values)

In [ ]:
data.term.unique()

In [ ]:
data.drop('grade', axis=1, inplace=True)

address
We are going to feature engineer a zip code column from the address in the data set. Create a column called 'zip_code' that extracts the zip code from the address column.

In [ ]:
data.address.head()

In [ ]:
data['zip_code'] = data.address.apply(lambda x: x[-5:])

In [ ]:
data.zip_code.value_counts()

In [ ]:
# data = pd.get_dummies(data, columns=['zip_code'], drop_first=True)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the zip_code column
data['zip_code_encoded'] = label_encoder.fit_transform(data['zip_code'])

label_encoder = LabelEncoder()
data['home_ownership_encoded'] = label_encoder.fit_transform(data['home_ownership'])

label_encoder = LabelEncoder()
data['purpose_encoded'] = label_encoder.fit_transform(data['purpose'])

data.drop('zip_code', axis=1, inplace=True)
data.drop('purpose', axis=1, inplace=True)
data.drop('home_ownership', axis=1, inplace=True)

data.drop('address', axis=1, inplace=True)

sub_grade_mapping = {
    'A1': 35, 'A2': 34, 'A3': 33, 'A4': 32, 'A5': 31,
    'B1': 30, 'B2': 29, 'B3': 28, 'B4': 27, 'B5': 26,
    'C1': 25, 'C2': 24, 'C3': 23, 'C4': 22, 'C5': 21,
    'D1': 20, 'D2': 19, 'D3': 18, 'D4': 17, 'D5': 16,
    'E1': 15, 'E2': 14, 'E3': 13, 'E4': 12, 'E5': 11,
    'F1': 10, 'F2': 9, 'F3': 8, 'F4': 7, 'F5': 6,
    'G1': 5, 'G2': 4, 'G3': 3, 'G4': 2, 'G5': 1
}
data['sub_grade_encoded'] = data['sub_grade'].map(sub_grade_mapping)
data.drop('sub_grade', axis=1, inplace=True)


Removing `issue_d` feature
This would be data leakage because we would not know the loan issue date beforehand when using our model/in our usecase. Therefore choosing to drop this feature.

In [ ]:
data.drop('issue_d', axis=1, inplace=True)

In [ ]:
data['earliest_cr_line'] = data['earliest_cr_line'].apply(lambda x: datetime.strptime(x, "%b-%Y").year)

In [ ]:
data.earliest_cr_line.nunique()

In [ ]:
data.earliest_cr_line.value_counts()

### Removing or Imputing missing values

In [ ]:
for column in data.columns:
    if data[column].isna().sum() != 0:
        missing = data[column].isna().sum()
        portion = (missing / data[column].shape[0]) * 100
        print(f"'{column}' number of missing values: '{missing}' ==> '{portion:.3f}%'")
        # print(
        #     f"{bold_start}'{column}'{bold_end}: number of missing values: {bold_start}'{missing}'{bold_end} ==> {bold_start}'{portion:.3f}%'{bold_end}")

In [ ]:
data['emp_title'].nunique()

The **emp_title** has too many unique values to encode and dataset will not be sufficent if used with it. So, the best thing to do is drop it.

In [ ]:
data.drop('emp_title', axis=1, inplace=True)

Now, let's look at **emp_length**

In [ ]:
data["emp_length"].unique()

In [ ]:
for year in data["emp_length"].unique():
    print(f"{year} years in the positions:")
    # print(f"{bold_start}{year}{bold_end} years in the positions:")
    print(f"{data[data.emp_length == year].loan_status.value_counts(normalize=True)}")
    print("\n=============================================\n")

**Charge offs** seem to be extremely similar across all **emp_length**. So, we are going to drop it.

In [ ]:
data.drop('emp_length', axis=1, inplace=True)

Now, let's have a look at **title**.

In [ ]:
data.title.value_counts().head()

In [ ]:
data.title.unique().shape

In [ ]:
data.drop("title", axis=1, inplace=True)

Let's have a look at **mort_acc**.

In [ ]:
data.mort_acc.value_counts()

In [ ]:
data.mort_acc.isna().sum()

Imputing **mort_acc** should be imputed using either mean or prediction model, for this purpose we will do using **Random Forest**

In [ ]:
# Split the data into two parts: one with missing values and one without
missing_data = data[data['mort_acc'].isnull()]
complete_data = data.dropna(subset=['mort_acc'])

# Select features and target variable
X = complete_data.drop(columns=['mort_acc'])
y = complete_data['mort_acc']

# Split the complete data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict missing values
imputed_values = model.predict(missing_data.drop(columns=['mort_acc']))

# Impute missing values in the mort_acc column
data.loc[data['mort_acc'].isnull(), 'mort_acc'] = imputed_values

In [ ]:
data.mort_acc.value_counts()

In [ ]:
for column in data.columns:
    if data[column].isna().sum() != 0:
        missing = data[column].isna().sum()
        portion = (missing / data.shape[0]) * 100
        print(f"'{column}': number of missing values '{missing}' ==> '{portion:.3f}%'")

# drop missing value instances that are less than 0.2% of the dataset
data.dropna(inplace=True)
display(data.shape)
data.loan_status.value_counts()

Removing `revol_util & pub_rec_bankruptcies` missing values instances
These two features have missing data points, but they account for less than 0.5% of the total data. So we are going to remove the rows that are missing those values in those columns with dropna().

In [ ]:
data.info()

## Preparing the Test data

### Checking distribution of data

In [ ]:
# Define a threshold for Z-score
threshold = 3

# Create a copy of the data to avoid modifying the original DataFrame
data_clean = data.copy()

# Dictionary to store counts of outliers for each column
outliers_count_dict = {}

# Loop through each numerical column in the DataFrame
for column in data_clean.select_dtypes(include=[np.number]).columns:
    # Calculate Z-scores
    z_scores = (data_clean[column] - data_clean[column].mean()) / data_clean[column].std()
    # Identify outliers
    outliers = np.abs(z_scores) > threshold
    # Store the count of previous outliers
    outliers_count_dict[column] = outliers.sum()
    # Calculate the mean value of the column
    mean_value = data_clean[column].mean()
    # If the column is of type int64, cast the mean value to int
    if pd.api.types.is_integer_dtype(data_clean[column]):
        mean_value = int(mean_value)
    # Replace outliers with the mean value of the column
    data_clean.loc[outliers, column] = mean_value

# Print the shape of the original and cleaned data
print(f"Original data shape: {data.shape}")
print(f"Cleaned data shape: {data_clean.shape}")

# Print count of previous outliers
print("\nCount of previous outliers in each column:")
for column, count in outliers_count_dict.items():
    print(f"Column: {column} - Outliers: {count}")

# Optional: Check if there are any remaining outliers
remaining_outliers = ((data_clean - data_clean.mean()) / data_clean.std()).abs() > threshold
print("\nRemaining outliers in each column after cleaning:")
print(remaining_outliers.sum())

## Oversampling the dataset using SMOTE and ADASYN 

In [ ]:
def smote_oversampling(x, y):
    """
    Synthetic Minority Sampling
    :param x: predictors dataframe
    :param y: label dataframe
    :return: x, y dataframes with synthetic instances
    """
    smote = SMOTE(random_state=42)
    # fit and resample the data
    x_resampled, y_resampled = smote.fit_resample(x, y)

    return x_resampled, y_resampled


def adasyn_oversampling(x, y):
    """
    Adaptive Synthetic Sampling
    :param x: predictors dataframe
    :param y: label dataframe
    :return: x, y dataframes with synthetic instances
    """
    adasyn = ADASYN(random_state=42)
    # fit and resample the data
    x_resampled, y_resampled = adasyn.fit_resample(x, y)

    return x_resampled, y_resampled


# Assuming data_clean is your DataFrame and 'target' is the target variable
X = data_clean.drop(columns=['loan_status'])
y = data_clean['loan_status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

smote_x, smote_y = smote_oversampling(X_train, y_train)
adasyn_x, adasyn_y = adasyn_oversampling(X_train, y_train)


print(f"Original dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_train_res)}")
print(f"SMOTE sampled dataset shape: {Counter(smote_y)}")
print(f"ADASYN sampled dataset shape: {Counter(adasyn_y)}")


## Feature Importance Analysis

In [ ]:
# print("Feature importance on SMOTE dataset")
# feature_importance_random_forest(smote_x, smote_y)

# print("Feature importance on ADASYN dataset")
# feature_importance_random_forest(adasyn_x, adasyn_y)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_train, X_val, y_train, y_val = train_test_split(smote_x, smote_y, test_size=0.25, random_state=42)

model = LogisticRegression(solver='liblinear', max_iter=1000)
# model = LogisticRegression()
model.fit(X_train, y_train)

y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_conf_matrix = confusion_matrix(y_val, y_val_pred)
val_class_report = classification_report(y_val, y_val_pred)

print(f'Validation Accuracy: {val_accuracy}')
print('Validation Confusion Matrix:')
print(val_conf_matrix)
print('Validation Classification Report:')
print(val_class_report)

In [ ]:
y_test_pred = model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_conf_matrix = confusion_matrix(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred)

print(f'Test Accuracy: {test_accuracy}')
print('Test Confusion Matrix:')
print(test_conf_matrix)
print('Test Classification Report:')
print(test_class_report)


In [ ]:
# Predict on validation set
y_val_pred = model.predict(X_val)

# Calculate metrics for validation set
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred)
val_recall = recall_score(y_val, y_val_pred)
val_f1_score = f1_score(y_val, y_val_pred)

# Fill metrics for validation set
metrics_dict_val['Logistic Regression'] = {'accuracy': val_accuracy, 'precision': val_precision, 'recall': val_recall, 'f1_score': val_f1_score}

# Predict on test set
y_test_pred = model.predict(X_test)

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1_score = f1_score(y_test, y_test_pred)

# Fill metrics for test set
metrics_dict_test['Logistic Regression'] = {'accuracy': test_accuracy, 'precision': test_precision, 'recall': test_recall, 'f1_score': test_f1_score}

# ANN

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout

# # Define the model
# input_dim = smote_x.shape[1]
# model = Sequential()
# model.add(Dense(64, input_dim=input_dim))
# model.add(Dropout(0.5))  # Dropout layer with 50% dropout rate
# model.add(Dense(32))
# model.add(Dropout(0.5))  # Dropout layer with 50% dropout rate
# model.add(Dense(16))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model with class weights
# model.fit(smote_x, smote_y, epochs=10, batch_size=32, validation_split=0.2)

# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f'Test Accuracy: {accuracy:.4f}')

# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)

# print(classification_report(y_test, y_pred_classes))

### XGBoost Classifier

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(smote_x, label=smote_y)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define the parameter dictionary
params = {
    'objective': 'binary:logistic',  # binary classification
    'max_depth': 10,  # maximum depth of a tree
    'eta': 0.03,  # step size shrinkage
    'eval_metric': 'logloss',  # evaluation metric
    'random_state': 42  # seed for reproducibility
}

# Specify the training and validation sets
evals = [(dtrain, 'train'), (dval, 'eval')]

# Train the model
bst = xgb.train(params, dtrain, num_boost_round=1000, evals=evals, early_stopping_rounds=10)

# Predict on the test set
y_pred_prob = bst.predict(dtest)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate metrics for validation set
y_val_pred = (bst.predict(dval) > 0.5).astype(int)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_score_val = f1_score(y_val, y_val_pred)

# Fill metrics for validation set
metrics_dict_val['XGBoost'] = {'accuracy': accuracy_val, 'precision': precision_val, 'recall': recall_val, 'f1_score': f1_score_val}

# Calculate metrics for test set
accuracy_test = accuracy_score(y_test, y_pred)
precision_test = precision_score(y_test, y_pred)
recall_test = recall_score(y_test, y_pred)
f1_score_test = f1_score(y_test, y_pred)

# Fill metrics for test set
metrics_dict_test['XGBoost'] = {'accuracy': accuracy_test, 'precision': precision_test, 'recall': recall_test, 'f1_score': f1_score_test}

print(classification_report(y_test, y_pred))

## 6. Evaluation of Metrics

In [ ]:
# Plot the metrics
models = list(metrics_dict_val.keys())
x = np.arange(len(models))  # the label locations
width = 0.35  # the width of the bars

# Create 4 subplots (one for each metric) arranged in a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

metrics = list(metrics_dict_val[models[0]].keys())  # Metrics names

for i, metric in enumerate(metrics):
    row = i // 2
    col = i % 2
    values_val = [metrics_dict_val[model][metric] for model in models]
    values_test = [metrics_dict_test[model][metric] for model in models]

    axs[row, col].bar(x - width / 2, values_val, width, label='Validation', color='b')
    axs[row, col].bar(x + width / 2, values_test, width, label='Test', color='g')

    axs[row, col].set_ylabel(metric.capitalize())
    axs[row, col].set_title(f'{metric.capitalize()} Comparison: Validation vs Test Set')
    axs[row, col].set_xticks(x)
    axs[row, col].set_xticklabels(models)
    axs[row, col].legend(loc='center right')

    # Attach a text label above each bar in rects, displaying its height.
    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            axs[row, col].annotate(f'{height:.2f}',
                            xy=(rect.get_x() + rect.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom')

    autolabel(axs[row, col].bar(x - width / 2, values_val, width, label='Validation', color='b'))
    autolabel(axs[row, col].bar(x + width / 2, values_test, width, label='Test', color='g'))

fig.tight_layout()

plt.show()
